## Protocol Analysis

It would probably be good to make these actual python libraries. Oh well.

In [ ]:
# First extract logs with "PERF" from the client/receiver logs

from pathlib import Path

filename="test.log"

if not Path(filename).exists():
    !cat ../logs/replica*.log ../logs/client*.log | grep PERF >{filename}

In [ ]:
# Basic log parsing

import re
import datetime 

def parse_time(line):
    match = re.search(f"([0-9]*:[0-9]*:[0-9]*.[0-9]*)", line)
    time_str = match.group(1);
    return datetime.datetime.strptime(time_str, "%H:%M:%S.%f")

def parse_tags(line):
    tags = {}
    line = line.split("PERF ")[1]
    for token in line.split():
        [tag, value] = token.split("=")
        tags[tag] = value
        try:
            tags[tag] = int(value)
        except ValueError as verr:
            pass
            
    return tags
        
def parse_line(line):
    time = parse_time(line)
    tags = parse_tags(line)

    tags["time"] = time
    return tags 

In [ ]:
events = []

with open(filename) as f:
    for line in f:
        events.append(parse_line(line))

In [ ]:
events = sorted(events, key=lambda x: x['time'])
commits = filter(lambda x: x["event"] == "commit", events)


In [ ]:
# Look at each client commit, make sure there are no two clients ops on the same seq
from collections import Counter

counts = Counter(c["seq"] for c in commits)
for seq in counts:
    if counts[seq] > 1:
        print(f"Sequence {seq} has different commits!")

In [ ]:
# Ensure all client operations are committed
n_clients = 2

for c_id in range(n_clients):
    c_commits = filter(lambda x: x["client_id"] == c_id, commits)
    seq = sorted(c["client_seq"] for c in c_commits)

    for x, y in zip(seq, seq[1:]):
        if (y-x) > 1:
            print(f"Client missed commits between {x}-{y}")
